In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from pathlib import Path

from dosemetrics.io import load_dose, load_mask
from dosemetrics.metrics.dvh import compute_dvh
from dosemetrics.metrics.scores import compute_conformity_index
from dosemetrics.utils.plotting import plot_dvh

## 1. Load Sample Data

Load a dose distribution and structure mask:

In [ ]:
# Load dose
dose = load_dose("path/to/dose.nii.gz")
print(f"Dose shape: {dose.shape}")
print(f"Dose range: {dose.min():.2f} - {dose.max():.2f} Gy")

# Load structure
ptv = load_mask("path/to/ptv.nii.gz")
print(f"PTV volume: {ptv.sum()} voxels")

## 2. Compute DVH

In [ ]:
# Compute DVH
dvh = compute_dvh(dose, ptv, organ_name="PTV")
print(dvh.head())

## 3. Visualize DVH

In [ ]:
# Create interactive plot
fig = plot_dvh(dvh, title="PTV Coverage")
fig.show()

## 4. Compute Quality Metrics

In [ ]:
# Conformity index
ci = compute_conformity_index(dose, ptv, prescription_dose=60.0)
print(f"Conformity Index: {ci:.3f}")

## 5. Export Results

In [ ]:
# Save DVH to CSV
dvh.to_csv("dvh_results.csv", index=False)
print("Results saved!")

## Try the Live Demo

Want to experiment without setting up data? Try our [interactive demo on Hugging Face](https://huggingface.co/spaces/contouraid/dosemetrics)!